This file contains the core components of the algorithm. It trains the MLKNN model using features selected by a genetic algorithm over 100 generations (modifiable as needed). The genetic algorithm is applied to both the filtered and original sets of features, aiming to minimize the number of features and the hamming loss. The objective values are then saved into a pickle file witht he oprion to save all feature populations for future evaluations and inspection.

In [ ]:
pip install pandas numpy scikit-multilearn scipy pymoo matplotlib scikit-learn minepy

In [ ]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from scipy.sparse import lil_matrix
from sklearn.metrics import hamming_loss
from pymoo.core.problem import Problem
import time
from pymoo.optimize import minimize
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.algorithms.soo.nonconvex.ga import GA
from pymoo.operators.sampling.rnd import BinaryRandomSampling
from pymoo.operators.crossover.ux import UniformCrossover
from pymoo.operators.mutation.bitflip import BitflipMutation
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from pymoo.indicators.hv import HV
from pymoo.config import Config
Config.warnings['not_compiled'] = False
from sklearn.feature_selection import mutual_info_classif, mutual_info_regression
from pymoo.util.nds.non_dominated_sorting import NonDominatedSorting
from minepy import pstats, cstats
from skmultilearn.adapt import MLkNN
from random import randrange
from pymoo.core.sampling import Sampling
from pymoo.core.duplicate import ElementwiseDuplicateElimination
import pickle
from pymoo.core.crossover import Crossover
import os
from pymoo.core.mutation import Mutation

The following code extracts the information obtained in the initial filteration, saved in the 'sorted_data.pkl' file. Only the indices of the features in the top front (ie. rank 0) are added to the 'tops' array. This might need modifications depending on how the data is stored.

In [ ]:
# Open the pickle file in read-binary mode and load the data
with open('sorted_data.pkl', 'rb') as f:
    sorted_data = pickle.load(f)

tops = []
# Print the loaded data to verify
for name, sorted_indices in sorted_data:
    print(f"Dataset name: {name}")
    tops.append(sorted_indices[0])

print(tops[0]) # replace witht he index of the current datset

The MyProblem class defines a multi-objective optimization problem for feature selection in a multi-label classification task using the MLkNN algorithm. It evaluates different feature subsets based on the number of selected features and the Hamming loss on validation data.

In [ ]:
class MyProblem(Problem):
  def __init__(self, xtrain, ytrain_lil, xvalidate, yvalidate_lil):
      super().__init__(n_var=xtrain.shape[1], n_obj=2, n_ieq_constr=1, xl=np.zeros(xtrain.shape[1]), xu=np.ones(xtrain.shape[1]))
      self.xtrain = xtrain
      self.ytrain_lil = ytrain_lil
      self.xvalidate = xvalidate
      self.yvalidate_lil = yvalidate_lil
      self.feature_names = xtrain.columns.tolist()
      self.total_features = xtrain.shape[1]  # Store the total number of features

  def _evaluate(self, x, out, *args, **kwargs):
      f1 = []
      f2 = []

      for bitstring in x:
          featureNames = [self.feature_names[i] for i, bit in enumerate(bitstring) if bit == 1]
          if not featureNames:
              ham_loss = 1  # High hamming loss if no features are selected
              num_features = 0
          else:
              trainx = self.xtrain[featureNames]
              classifier = MLkNN(k=3)
              classifier.fit(trainx, self.ytrain_lil)

              validatex = self.xvalidate[featureNames]

              predy = classifier.predict(validatex)
              ham_loss = hamming_loss(self.yvalidate_lil.toarray(), predy.toarray())
              num_features = len(featureNames)

          # Calculate the ratio of selected features to total features
          feature_ratio = num_features / self.total_features

          f1.append(feature_ratio)
          f2.append(ham_loss)  # Hamming loss is already a value between 0 and 1

      # Constraint: At least one feature must be selected
      g1 = [1 - sum(bitstring) for bitstring in x]  # Constraint that at least one feature must be selected

      out["F"] = np.column_stack((f1, f2))
      out["G"] = np.column_stack(g1)

The algorithm defines the NSGA-II optimization algorithm for solving the multi-objective feature selection problem. It initializes a population, performs binary uniform crossover and bit-flip mutation, and eliminates duplicate solutions to evolve optimal feature subsets.

BinaryRandomSampling Class:
The BinaryRandomSampling class is designed to generate an initial population for a binary optimization problem. Each individual in the population is represented by a binary string, where each bit indicates the presence (1) or absence (0) of a feature. This class ensures that each individual in the population is unique.

BinaryUniformCrossover Class:
The BinaryUniformCrossover class implements a uniform crossover operator for binary genetic algorithms. It creates offspring by combining features from two parent individuals based on a specified crossover probability.

BinaryBitflipMutation Class:
The BinaryBitflipMutation class implements a bit-flip mutation operator for binary genetic algorithms. It mutates individuals by flipping each bit with a specified probability.

create_nsga2_algorithm Function:
The create_nsga2_algorithm function creates an instance of the NSGA2 algorithm for multi-objective optimization. It uses the previously defined BinaryRandomSampling, BinaryUniformCrossover, and BinaryBitflipMutation classes to define the algorithm's behavior.

In [ ]:
class BinaryRandomSampling(Sampling):
    def __init__(self, num_features, pop_size=100):
        self.num_features = num_features
        self.pop_size = pop_size
        super().__init__()

    def _do(self, problem, n_samples, **kwargs):
        population = []
        seen = set()
        feature_counts = []
        population.append(np.ones(self.num_features, dtype=int))
        while len(population) < self.pop_size: # keep regenerating unique individuals
            rn = randrange(0, self.num_features + 1, 1)  # +1 to include all features
            arr = np.zeros(self.num_features, dtype=int)
            arr[:rn] = 1
            np.random.shuffle(arr)
            arr_tuple = tuple(arr)
            if arr_tuple not in seen:
                population.append(arr)
                seen.add(arr_tuple)
                feature_counts.append(rn)
            else:
                print("Duplicate found during initialization and ignored:", arr_tuple)

        # Plotting to verify the distribution is uniform
        plt.figure(figsize=(6, 4))  # Adjust the figure size as needed
        plt.hist(feature_counts, bins='auto', color='blue', alpha=0.7, rwidth=0.75)
        plt.grid(axis='y', alpha=0.75)
        plt.xlabel('Number of Features', fontsize=9)  # Adjust font size
        plt.ylabel('Frequency', fontsize=9)  # Adjust font size
        plt.title('Distribution of Number of Features (Verification of Initialization)', fontsize=11)  # Adjust font size
        plt.savefig("Histogram_of_Initial_Population.eps", format='eps')
        plt.show()
        print()
        return np.array(population)

#####################################################################################################

class BinaryUniformCrossover(Crossover):

    def __init__(self, prob=0.5):
        super().__init__(2, 2)
        self.prob = prob

    def _do(self, _, X, **kwargs):
        # print("using the manually defined crossover")
        _, n_matings, n_var = X.shape
        M = np.random.random((n_matings, n_var)) < self.prob
        offspring = np.zeros_like(X)
        for i in range(n_matings):
            for j in range(n_var):
                if M[i, j]:
                    offspring[0, i, j] = X[1, i, j]
                    offspring[1, i, j] = X[0, i, j]
                else:
                    offspring[0, i, j] = X[0, i, j]
                    offspring[1, i, j] = X[1, i, j]
        return offspring

#####################################################################################################

class BinaryBitflipMutation(Mutation):
    def __init__(self, prob=0.01, **kwargs):
        super().__init__(**kwargs)
        self.prob = prob

    def _do(self, problem, X, **kwargs):
        # print("using the manually defined mutation")
        prob_var = self.get_prob_var(problem, size=(len(X), 1))
        Xp = np.copy(X)
        flip = np.random.random(X.shape) < self.prob * prob_var
        Xp[flip] = ~X[flip]
        return Xp

#####################################################################################################

def create_nsga2_algorithm(total_features):
    return NSGA2(
        pop_size=100,
        sampling=BinaryRandomSampling(total_features),
        crossover=BinaryUniformCrossover(prob=0.9),
        mutation=BinaryBitflipMutation(prob=0.01),
        eliminate_duplicates=True
    )

The plot_final_pareto_front function visualizes the final Pareto front obtained from the NSGA2 optimization process. This plot helps in understanding the trade-off between the number of selected features and the validation error (Hamming loss) for the multi-label classification problem.

In [ ]:
def plot_final_pareto_front(pop, f, total_features, type, dataset, run_num): # this is the output of nsga2 directly (ie. the validation)
    plt.figure(figsize=(6, 4))
    actual_features_pop = pop.get("F")[:,0] * total_features
    actual_features_f = f[:,0] * total_features
    plt.scatter(actual_features_pop, pop.get("F")[:,1], edgecolor="blue", facecolor="none", label="Solutions")
    plt.scatter(actual_features_f, f[:,1], marker='*', edgecolor="red", facecolor="none", label="Non-dominated Pareto Front")
    plt.xlabel('Number of Features')
    plt.ylabel('Validation Error (Hamming Loss)')
    plt.legend()
    plt.title(f"Final Pareto Front of the {dataset} dataset with {type} data on validation")
    
    # Save the figure
    filename = f"Final_Pareto_Front_with_{type}_data_on_validation_run {run_num}.eps"
    plt.savefig(filename, format='eps')
    plt.show()
    print("\n")

The calculate_metrics function computes several performance metrics for the Pareto front obtained from the NSGA2 optimization algorithm.

In [ ]:
def calculate_metrics(res, xtrain, ytrain_lil, xvalidate, yvalidate_lil, xtest, ytest_lil, total_features, type, dataset, run_num):
    pop = res.pop
    F = res.F

    # Find the solution with the minimum Hamming loss on the validation set, F[:, 1] holds the values of the hamming losses
    min_hl_val_idx = np.argmin(F[:, 1])

    # extracting the minimum hamming loss value as well as the numebr of features used to obtain this value
    min_HL_val = F[min_hl_val_idx, 1]
    min_f_val = F[min_hl_val_idx, 0] * total_features # adjust to include the number of features not the scaling

    # Calculate the hamming loss of the final pareto front on the test set
    num_features = []
    hl_test = []
    for i in range(len(pop)):
        selected_features = np.where(pop[i].get("X") == 1)[0]
        if len(selected_features) == 0:
            ham_loss = 1.0  # If no features selected, set a high hamming loss
        else:
            model = MLkNN(k=3)
            model.fit(xtrain.iloc[:, selected_features], ytrain_lil)
            predy = model.predict(xtest.iloc[:, selected_features])
            ham_loss = hamming_loss(ytest_lil.toarray(), predy.toarray())

        num_features.append(len(selected_features))
        hl_test.append(ham_loss)

    # Convert test results to numpy array for plotting
    test_F = np.column_stack((num_features, hl_test))

    # Non-dominated sorting to find the Pareto-optimal solutions in the test set
    nds = NonDominatedSorting()
    pareto_optimal_indices_test = nds.do(test_F, only_non_dominated_front=True)
    pareto_optimal_test_F = test_F[pareto_optimal_indices_test]

    # Scale the objective values using MinMaxScaler
    scaler = MinMaxScaler()
    pareto_optimal_test_F_scaled = scaler.fit_transform(pareto_optimal_test_F)

    # Define the reference point
    ref_point = np.array([1.1, 1.1]) # if we have (1,1) will arise a probelm if the paretofront has only 1 solution at the point (1,1)

    # Calculate hypervolume for Pareto-optimal solutions in the test set
    hv_indicator_test = HV(ref_point=ref_point)
    HV_test = hv_indicator_test(pareto_optimal_test_F_scaled)

    # Calculate hypervolume for Pareto-optimal solutions in the validation set
    pareto_optimal_val_F = F  # The F already contains the Pareto-optimal front
    ref_point_val = np.array([1.1, 1.1])  # The referenc epoint shouldnt be 1,1, 1.1 is ebtter
    pareto_optimal_val_F_scaled = scaler.fit_transform(pareto_optimal_val_F) 
    hv_indicator_val = HV(ref_point=ref_point_val)
    HV_val = hv_indicator_val(pareto_optimal_val_F_scaled)

    # Find the solution with the minimum Hamming loss on the test set
    min_HL_test_idx = np.argmin(pareto_optimal_test_F[:, 1])
    min_HL_test = pareto_optimal_test_F[min_HL_test_idx, 1]
    min_f_test = pareto_optimal_test_F[min_HL_test_idx, 0]

    # Plot Hamming loss values for test set of solutions in optimal Pareto front
    plt.figure(figsize=(6, 4))
    plt.scatter(test_F[:, 0], test_F[:, 1], edgecolor="blue", facecolor="none", label="Solutions")
    plt.scatter(pareto_optimal_test_F[:, 0], pareto_optimal_test_F[:, 1], marker='*', edgecolor="red", facecolor="none", label="Non Dominated Pareto Front")
    plt.xlabel('Number of Features')
    plt.ylabel('Hamming Loss (Test)')
    plt.legend()
    plt.title(f"Final Pareto Front of the {dataset} dataset with {type} data on test")
    plt.grid(False)

    # Save the figure
    filename = f"Final_Pareto_Front_with_{type}_data_on_test_run {run_num}.eps"
    plt.savefig(filename, format='eps')
    plt.show()

    print("\n")

    return  min_HL_val, min_f_val, min_HL_test, min_f_test, HV_val, HV_test, hl_test


In the section below, load the datasets from the ARFF files. The dataframe is then split into training, validation, and test sets, allocating 50%, 20%, and 30% of the data, respectively. The labels and features are separated accordingly. The labels are converted into a lil_matrix format for compatibility with the MLkNN algorithm.

In [ ]:
data_matrix, feature_names, labels = parse_arff_data_name('name.arff') # replace with dataset name and reading function

# Convert the data matrix into a DataFrame
df = pd.DataFrame(data_matrix, columns=feature_names + labels)

# Separate the labels and features in the DataFrame
y = df[labels]
X = df.drop(columns=labels)

print(f"The shape of the data matrix is {data_matrix.shape} with {len(feature_names)} features and {len(labels)} labels. \n")

xtrain_validate, xtest, ytrain_validate, ytest = train_test_split(X, y, test_size=0.3, random_state = 42)
xtrain, xvalidate, ytrain, yvalidate = train_test_split(xtrain_validate, ytrain_validate, test_size=0.2, random_state = 42)

ytrain_lil = lil_matrix(ytrain)
yvalidate_lil = lil_matrix(yvalidate)
ytest_lil = lil_matrix(ytest)

The classifer is train with k = 3 and hamming loss values are obtained for both the validation and test tests. 

In [ ]:
classifier_all_features = MLkNN(k=3)
classifier_all_features.fit(xtrain, ytrain_lil)

# Predict labels for the validation set using all features
predy_all_features_validate = classifier_all_features.predict(xvalidate)

# Predict labels for the test set using all features
predy_all_features_test = classifier_all_features.predict(xtest)

# Calculate Hamming loss for validation set when all features are used
ham_loss_all_features_validate = hamming_loss(yvalidate_lil.toarray(), predy_all_features_validate.toarray())

# Calculate Hamming loss for test set when all features are used
ham_loss_all_features_test = hamming_loss(ytest_lil.toarray(), predy_all_features_test.toarray())

print(f"The Hamming loss resulting on the validation set using all {xtrain.shape[1]} features is {ham_loss_all_features_validate}")
print(f"The Hamming loss resulting on the test set using all {xtrain.shape[1]} features is {ham_loss_all_features_test}")


The follwoing code section filters the dataset to include only features obtained from the initial filteration technique. Then, the classifer is train with k = 3 and hamming loss values are obtained for both the validation and test tests. 

In [ ]:
top_front = tops[0] # replace with the index of the current dataset

print(f"There are {len(top_front)} features in the top front which are being used in the second phase.")

# Extract the selected features from xtrain_validate using iloc
selected_features_phase1 = xtrain_validate.iloc[:, top_front]

# Get the column names of the selected features
selected_feature_names_phase1 = selected_features_phase1.columns.tolist()

# Select the features from the training, validation, and test sets
xtrain_filtered = xtrain.iloc[:, top_front]
xvalidate_filtered = xvalidate.iloc[:, top_front]
xtest_filtered = xtest.iloc[:, top_front]

# Instantiate the MLkNN classifier with k=3
model = MLkNN(k=3)

# Fit the model using the selected features from the training data
model.fit(xtrain_filtered, ytrain_lil)

# Predict labels for the validation data using the trained model and selected features
val_predy = model.predict(xvalidate_filtered)

# Predict labels for the test data using the trained model and selected features
test_predy = model.predict(xtest_filtered)

# Calculate the Hamming loss between the true labels and predicted labels for the validation data
ham_loss_all_features_from_phase_1_validate = hamming_loss(yvalidate_lil.toarray(), val_predy.toarray())

# Calculate the Hamming loss between the true labels and predicted labels for the test data
ham_loss_all_features_from_phase_1_test = hamming_loss(ytest_lil.toarray(), test_predy.toarray())

print(f"The Hamming loss resulting on the validation set using the {xtrain_filtered.shape[1]} filtered features is {ham_loss_all_features_from_phase_1_validate}.")
print(f"The Hamming loss resulting on the test set using the {xtrain_filtered.shape[1]} filtered features is {ham_loss_all_features_from_phase_1_test}.")

In [ ]:
roblem_not_filtered = MyProblem(xtrain, ytrain_lil, xvalidate, yvalidate_lil)
problem_filtered = MyProblem(xtrain_filtered, ytrain_lil, xvalidate_filtered, yvalidate_lil)

def run_optimization(problem, algorithm, n_gen):
    res = minimize(problem, algorithm, ('n_gen',n_gen), verbose=True, save_history=False)
    return res

pickle_files = []

# Determine the number of runs you want to obtain.
for j in range(5):
    print(f"Running the algorithm for hte {j} time. \n")

    res_not_filtered = run_optimization(problem_not_filtered, create_nsga2_algorithm(xtrain.shape[1]), 100) # 100 defines the number of generations the algorithm will run for.
    res_filtered = run_optimization(problem_filtered, create_nsga2_algorithm(xtrain_filtered.shape[1]), 100)

    # Plot final Pareto front for both res
    non_dominated_feature_subsets_not_filtered = plot_final_pareto_front(res_not_filtered.pop, res_not_filtered.F, xtrain.shape[1], 'unfiltered', 'name', j)
    non_dominated_feature_subsets_filtered = plot_final_pareto_front(res_filtered.pop, res_filtered.F, xtrain_filtered.shape[1], 'filtered', 'name', j)

    # Calculate metrics for unfiltered data
    min_HL_val_nf, min_f_val_nf, min_HL_test_nf, min_f_test_nf, HV_val_nf, HV_test_nf, hamming_losses_on_test_unfilitered = calculate_metrics(res_not_filtered, xtrain, ytrain_lil, xvalidate, yvalidate_lil, xtest, ytest_lil, xtrain.shape[1], 'unfiltered', 'name', j)

    # Calculate metrics for filtered data
    min_HL_val_f, min_f_val_f, min_HL_test_f, min_f_test_f, HV_val_f, HV_test_f,  hamming_losses_on_test_filtered = calculate_metrics(res_filtered, xtrain_filtered, ytrain_lil, xvalidate_filtered, yvalidate_lil, xtest_filtered, ytest_lil, xtrain_filtered.shape[1], 'filtered', 'name', j)

    pop_not_filtered = res_not_filtered.pop
    pop_filtered = res_filtered.pop

    f_not_filtered = res_not_filtered.F
    f_filtered = res_filtered.F

    hamming_loss_on_validation_not_filtered = pop_not_filtered.get("F")[:,1]
    hamming_loss_on_validation_filtered = pop_filtered.get("F")[:,1]

    total_features_not_filtered = xtrain.shape[1]
    total_features_filtered = xtrain_filtered.shape[1]

    ###############################################################################################
    # Save the data in a pickle file

    # Define your results dictionary with simpler keys
    results = {
        "Run number :": j,
        "HamLoss_Val_All": ham_loss_all_features_validate,
        "HamLoss_Test_All": ham_loss_all_features_test,
        "HamLoss_Val_Filtered": ham_loss_all_features_from_phase_1_validate,
        "HamLoss_Test_Filtered": ham_loss_all_features_from_phase_1_test,
        "MinHamLoss_Val_Unf": min_HL_val_nf,
        "NumFeatures_MinHamLoss_Val_Unf": min_f_val_nf,
        "MinHamLoss_Test_Unf": min_HL_test_nf,
        "NumFeatures_MinHamLoss_Test_Unf": min_f_test_nf,
        "MinHamLoss_Val_Filt": min_HL_val_f,
        "NumFeatures_MinHamLoss_Val_Filt": min_f_val_f,
        "MinHamLoss_Test_Filt": min_HL_test_f,
        "NumFeatures_MinHamLoss_Test_Filt": min_f_test_f,
        "HV_Val_Unf": HV_val_nf,
        "HV_Test_Unf": HV_test_nf,
        "HV_Val_Filt": HV_val_f,
        "HV_Test_Filt": HV_test_f
    }

    # Create a dictionary to store the combined results
    combined_results = {}

    # Copy the existing results to the combined results
    combined_results.update(results)

    # Add information for each individual in the unfiltered population
    for i, individual in enumerate(pop_not_filtered):
        x = individual.get("X")  # Decision variables (selected features)
        f = individual.get("F")  # Objective values (number of features ratio, validation error)

        # Multiply the first objective by the total number of features
        num_features = f[0] * total_features_not_filtered
        validation_error = f[1]

        # Get the corresponding Hamming loss on the test set
        test_error = hamming_losses_on_test_unfilitered[i]

        # Create a dictionary for each individual
        individual_dict = {
            "Selected Features": x,
            "Number of Features": num_features,
            "Hamming Loss on Validation Set": validation_error,
            "Hamming Loss on Test Set": test_error
        }

        # Add the individual's information to the combined results
        combined_results[f"Individual {i + 1} (Unfiltered Population)"] = individual_dict

    # Add information for each individual in the filtered population
    for i, individual in enumerate(pop_filtered):
        x = individual.get("X")  # Decision variables (selected features)
        f = individual.get("F")  # Objective values (number of features ratio, validation error)

        # Multiply the first objective by the total number of features
        num_features = f[0] * total_features_filtered
        validation_error = f[1]

        # Get the corresponding Hamming loss on the test set
        test_error = hamming_losses_on_test_filtered[i]

        # Create a dictionary for each individual
        individual_dict = {
            "Selected Features": x,
            "Number of Features": num_features,
            "Hamming Loss on Validation Set": validation_error,
            "Hamming Loss on Test Set": test_error
        }

        # Add the individual's information to the combined results
        combined_results[f"Individual {i + 1} (Filtered Population)"] = individual_dict

    # Save the combined results to a file (e.g., pickle)
    pickle_file = f"combined_results{j}.pkl"

    with open(f"combined_results{j}.pkl", "wb") as f:
        pickle.dump(combined_results, f)
    print("Combined results saved to combined_results.pkl")

    pickle_files.append(pickle_file)